In [ ]:
# Criando a estrutura de pastas 
!hdfs dfs -mkdir -p /user/raoni/

In [ ]:
# Testando se a criação aconteceu com sucesso
!hdfs dfs -ls /user/raoni/

Os seguintes comandos foram rodados no namenode:

hdfs dfs -put input/dados_covid/ /user/raoni/

ps: para acessaro o name node digite no terminal:
docker exec -it namenode bash

In [1]:
from pyspark.sql.types import StructType

In [2]:
dados_schema = StructType()\
    .add("regiao", "string")\
    .add("estado", "string")\
    .add("municipio", "string")\
    .add("coduf", "integer")\
    .add("codmun", "integer")\
    .add("codRegiaoSaude", "integer")\
    .add("nomeRegiaoSaude", "string")\
    .add("data", "timestamp")\
    .add("semanaEpi", "integer")\
    .add("populacaoTCU2019", "integer")\
    .add("casosAcumulado","integer")\
    .add("casosNovos","integer")\
    .add("obitosAcumulado","integer")\
    .add("obitosNovos","integer")\
    .add("Recuperadosnovos","integer")\
    .add("emAcompanhamentoNovos","integer")\
    .add("interior/metropolitana","integer")

In [3]:
dados = spark.read.csv("/user/raoni/dados_covid/*.csv", header="true", sep=";", schema=dados_schema)

In [4]:
dados.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: integer (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



In [ ]:
# Salvado como tibela hive particionada por município  
dados.write.saveAsTable("dados_tabela_hive", format="parquet", partitionBy="municipio")

In [5]:
!hdfs dfs -ls /user/hive/warehouse/dados_tabela_hive

Found 5299 items
-rw-r--r--   2 root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/_SUCCESS
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abadia de Goiás
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abadia dos Dourados
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abadiânia
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abaetetuba
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abaeté
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abaiara
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Abaré
drwxr-xr-x   - root supergroup          0 

drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boa Vista do Incra
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boa Vista do Ramos
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boa Vista do Sul
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boa Vista do Tupim
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boca da Mata
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Boca do Acre
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Bocaina
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Bocaina de Minas
drwxr-xr-x 

drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa do Mato
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa do Ouro
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa do Piauí
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa do Sítio
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa do Tocantins
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa dos Gatos
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa dos Patos
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Lagoa dos Três Cantos
drwxr

drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uraí
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Urbano Santos
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uru
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uruana
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uruana de Minas
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uruará
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Uruaçu
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hive/warehouse/dados_tabela_hive/municipio=Urubici
drwxr-xr-x   - root supergroup          0 2022-04-25 17:35 /user/hiv

In [6]:
dados_recife = spark.read.parquet("/user/hive/warehouse/dados_tabela_hive/municipio=Recife")

In [7]:
dados_recife.show(5)

+--------+------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|  regiao|estado|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|               data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+--------+------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Nordeste|    PE|   26|261160|         26010|         RECIFE|2021-01-01 00:00:00|       53|         1645727|         51088|       287|           2695|          0|            null|                 null|                     1|
|Nordeste|    PE|   26|261160|         26010|         RECIFE|2021-01-02 00:00:00|       53|         

In [8]:
dados_recife.agg({"obitosAcumulado":"sum"}).show()

+--------------------+
|sum(obitosAcumulado)|
+--------------------+
|             1159750|
+--------------------+



In [9]:
recife_obitos_acumulados = dados_recife.agg({"obitosAcumulado":"sum"})

In [10]:
recife_obitos_acumulados.show()

+--------------------+
|sum(obitosAcumulado)|
+--------------------+
|             1159750|
+--------------------+



In [11]:
recife_obitos_acumulados = recife_obitos_acumulados.withColumnRenamed("sum(obitosAcumulado)", "obitosAcumulados")

In [12]:
recife_obitos_acumulados.show()

+----------------+
|obitosAcumulados|
+----------------+
|         1159750|
+----------------+



In [ ]:
# Savando a primeira visualização como tabela hive
recife_obitos_acumulados.write.saveAsTable("obitos_aculumados_recife", format="parquet")

In [13]:
dados_caruaru_pe = spark.read.parquet("/user/hive/warehouse/dados_tabela_hive/municipio=Caruaru")

In [14]:
dados_caruaru_pe.agg({"obitosAcumulado":"sum"}).show()

+--------------------+
|sum(obitosAcumulado)|
+--------------------+
|              152359|
+--------------------+



In [15]:
caruaru_obitos_acumulados = dados_caruaru_pe.agg({"obitosAcumulado":"sum"})

In [16]:
caruaru_obitos_acumulados = caruaru_obitos_acumulados.withColumnRenamed("sum(obitosAcumulado)", "obitosAcumulados")

In [17]:
caruaru_obitos_acumulados.show()

+----------------+
|obitosAcumulados|
+----------------+
|          152359|
+----------------+



In [ ]:
# Salvando segundo visualização 
caruaru_obitos_acumulados.write.parquet("/user/raoni/obitos_acumulados_caruaru", compression="snappy")

In [18]:
dados_petrolina_pe = spark.read.parquet("/user/hive/warehouse/dados_tabela_hive/municipio=Petrolina")

In [19]:
dados_petrolina_pe.agg({"obitosAcumulado":"sum"}).show()

+--------------------+
|sum(obitosAcumulado)|
+--------------------+
|               68700|
+--------------------+



In [20]:
petrolina_obitos_acumulados = dados_petrolina_pe.agg({"obitosAcumulado":"sum"})

In [21]:
petrolina_obitos_acumulados = petrolina_obitos_acumulados.withColumnRenamed("sum(obitosAcumulado)", "obitosAcumulados")

In [22]:
petrolina_obitos_acumulados.show()

+----------------+
|obitosAcumulados|
+----------------+
|           68700|
+----------------+



In [ ]:
petrolina_obitos_acumulados.write.csv("/user/raoni/obitos_acumulados_petrolina", header='true')

In [23]:
!hdfs dfs -ls /user/raoni/obitos_acumulados_petrolina

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-25 18:19 /user/raoni/obitos_acumulados_petrolina/_SUCCESS
-rw-r--r--   2 root supergroup         23 2022-04-25 18:19 /user/raoni/obitos_acumulados_petrolina/part-00000-2366ae0d-2853-4209-b8a8-8509d8d8d384-c000.csv


Processos realizados no conteiner kafka:

docker exec -it kafka bash

=> Criação do tópico
kafka-topics.sh --bootstrap-server kafka:9092 --topic topicVisualizacao --create --partitions 1 --replication-factor 1

=> Confirmando se a criação aconteceu com sucesso
kafka-topics.sh --bootstrap-server kafka:9092 --list

=> Enviar a visualização obitos_acumulados_petrolina para o container kafka
no namenode: 

hdfs dfs -get /user/raoni/obitos_acumulados_petrolina
mv obitos_acumulados_petrolina/ input/

Na pasta input

docker cp obitos_acumulados_petrolina kafka:/

=> Criar um produtor enviando um arquivo:
kafka-console-producer.sh --broker-list kafka:9092 --topic topicVisualizacao < part-00000-2366ae0d-2853-4209-b8a8-8509d8d8d384-c000.csv 
